In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


import numpy as np

In [3]:
class camera:
    def __init__(self, index, mode = "dev"):
        self.cap = cv2.VideoCapture(index)
        self.face_mesh = mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5)     
        self.success, self.image = self.cap.read()
        self.results = self.face_mesh.process(self.image)
        self.loop = 0
        self.dir_vector = np.array([0.0,0.0,0.0])
        self.dir_state = 0
        self.mode = mode
        self.face_loc = np.array([0.0,0.0,0.0])
        self.eye = [0,0]
        
    def camera_update(self):
        self.success, self.image = self.cap.read()
    
    def get_face_mesh_data(self):
        self.image.flags.writeable = False
        self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        self.results = self.face_mesh.process(self.image)
    
    def draw_face_mesh_data(self):
        self.image.flags.writeable = True
        self.image = cv2.cvtColor(self.image, cv2.COLOR_RGB2BGR)
        if self.results.multi_face_landmarks:
            for face_landmarks in self.results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=self.image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=self.image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=self.image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
                # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(self.image, 1))
    
    def get_data(self):
        for face_landmarks in self.results.multi_face_landmarks:
            result = face_landmarks.landmark
        return result
    
    def release(self):
        self.cap.release()
    
    def calculate_face_dir_vector(self,data):
        # 143 : right eye end , 272 : left eye end , 199 : jaw end
        if(self.mode == "dev"):
            print("143 : ",[data[143].x,data[143].y,data[143].z]," 272: ",[data[272].x,data[272].y,data[272].z]," 199 : ",[data[199].x,data[199].y,data[199].z])
        vector_a = np.array([data[199].x,data[199].y,data[199].z]) - np.array([data[143].x,data[143].y,data[143].z])                          
        vector_b = np.array([data[272].x,data[272].y,data[272].z]) - np.array([data[199].x,data[199].y,data[199].z])
        if(self.mode == "dev"):
            print("vector a : ",vector_a," vector b : ",vector_b)
        result = np.cross(vector_a,vector_b) 
        if(self.mode == "dev"):
            print("origin result vector : ",result)
        return result / np.linalg.norm(result)
    
    def calculate_face_loc(self,data):
        return np.array([(data[143].x+data[199].x+data[272].x)/3,(data[143].z+data[199].z+data[272].z)/3,(data[143].y+data[199].y+data[272].y)/3])
    
    def check_eye(self,data):
        eye_right = np.array([data[159].x,data[159].y,data[159].z]) - np.array([data[145].x,data[145].y,data[145].z])                          
        eye_left = np.array([data[386].x,data[386].y,data[386].z]) - np.array([data[374].x,data[374].y,data[374].z])
        return [np.linalg.norm(eye_left, 2),np.linalg.norm(eye_right, 2)]
    
    def current_face_dir_state(self):
#         7,8,9
#         4,5,6       no face : -1
#         1,2,3
        if(self.dir_vector[0] == 0 and self.dir_vector[1] == 0 and self.dir_vector[2] == 0):
            self.dir_state = -1
            return
        self.dir_state = 5
        if(self.dir_vector[1] < -0.4):
            self.dir_state += 3
        elif(self.dir_vector[1] > 0.4):
            self.dir_state -= 3
        
        if(self.dir_vector[0] < -0.5):
            self.dir_state += 1
        elif(self.dir_vector[1] > 0.5):
            self.dir_state -= 1
    
    def current_face_dir_to_text(self):
        res = ""
        if(self.dir_state == -1):
            return "no face"
        
        if(self.dir_state % 3 == 0):
            res += "right "
        elif(self.dir_state % 1 == 1):
            res += "left "
            
        if(self.dir_state > 6):
            res += "up "
        elif(self.dir_state < 4):
            res += "down "
        return res
        
        
        
    def run(self):
        while self.cap.isOpened():
            self.camera_update()
            if not self.success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue
            
            self.get_face_mesh_data()
            
            self.draw_face_mesh_data()
            
            if(self.results.multi_face_landmarks == None):
                self.dir_vector = np.array([0,0,0])
                if(self.mode != "build"):
                    print("no face")
                continue
                
            
            print(self.loop)
            data = self.get_data()
            self.dir_vector = self.calculate_face_dir_vector(data)
            self.face_loc = self.calculate_face_loc(data)
            self.eye = self.check_eye(data)
            self.current_face_dir_state()

            if(self.mode != "build"):
                print(self.eye)
#                 print("direction : ",self.dir_state," ",self.current_face_dir_to_text())
                
            if(self.mode == "dev"):
                
                print(self.dir_vector)
                
            self.loop+=1
            if cv2.waitKey(5) & 0xFF == 27:
                break
        
        self.release()

In [ ]:
camera1 = camera(0, "only dir")
camera1.run()


# build / only dir / dev

# direction [0,0,-1] -> face front
# direction [-1,0,0] -> face right
# direction [1,0,0] -> face left
# direction [0,-1,0] -> face up
# direction [0,1,0] -> face down



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
0
[0.029083046730693878, 0.01776416911888467]
1
[0.03193045688649284, 0.014721688154825064]
2
[0.029462743724411423, 0.012336855888452184]
3
[0.03466732383166592, 0.011543328473099194]
4
[0.03307610275631562, 0.012401278243184401]
5
[0.029406341256962506, 0.017775583651818083]
6
[0.03786668682004302, 0.013623683376077625]
7
[0.03275661459044809, 0.018533154440152327]
8
[0.0292248081929259, 0.016177870119968935]
9
[0.03427236567545657, 0

163
[0.028632682207346948, 0.01615489191921464]
164
[0.027761032837645003, 0.017032099536365728]
165
[0.02553547884082259, 0.01640994993226713]
166
[0.02590668027103066, 0.017457150995843027]
167
[0.027639534930603973, 0.020048703767200426]
168
[0.026796588018348225, 0.018711842988819043]
169
[0.023285551488017153, 0.01809398222612808]
170
[0.0277799819745804, 0.011849688796400457]
171
[0.035992488314403416, 0.027365806077695124]
172
[0.039637119075648736, 0.047341843872750235]
173
[0.032344340246657395, 0.04715392703752451]
174
[0.03388508405466546, 0.04324310593460043]
175
[0.027651869710061422, 0.042294441774750446]
176
[0.02712524615113063, 0.04105634578941164]
177
[0.022579874734910056, 0.042042587493775485]
178
[0.031074754942480846, 0.04323035085028709]
179
[0.030888071830634772, 0.041668683738407604]
180
[0.02967096520386895, 0.04370644249266615]
181
[0.03063235581730809, 0.03552531172367829]
182
[0.03080197719300322, 0.03497582593620018]
183
[0.028068647974962443, 0.0384313837

338
[0.017092981179212225, 0.030450886737337]
339
[0.015694751462314947, 0.029243443089954965]
340
[0.017968224158034, 0.0296963065867407]
341
[0.018642941690201722, 0.03235722047735782]
342
[0.01587046136603775, 0.030859687068577536]
343
[0.01647800988838338, 0.03219989670340898]
344
[0.018551617786046838, 0.03042328205659672]
345
[0.01697516684073562, 0.03152088333565037]
346
[0.017498592038838398, 0.031042859088678373]
347
[0.01650220328620468, 0.030001955284690573]
348
[0.015951115517374125, 0.028879886922408473]
349
[0.01726951411070024, 0.029483528775185393]
350
[0.017856945582421967, 0.031966141160606844]
351
[0.01449023498883447, 0.03379425401038322]
352
[0.016640026846052366, 0.03391064029064479]
353
[0.015223234844533907, 0.032466903753526274]
354
[0.015761560249572898, 0.03104284726665526]
355
[0.01420953314984705, 0.0344614883484466]
356
[0.01501519438649441, 0.03364327367803676]
357
[0.014802561751202294, 0.031843417473898475]
358
[0.017481446751287603, 0.03209326715900647

514
[0.03472081712097571, 0.03326245444387958]
515
[0.033680532883838425, 0.03232464797672728]
516
[0.033576754188008394, 0.032564589414361717]
517
[0.0329407126256491, 0.033123857020042786]
518
[0.03348627637737279, 0.032713438153234416]
519
[0.03214128136885825, 0.03293793610043724]
520
[0.032902427353925325, 0.03210048014595892]
521
[0.03378539376512568, 0.032085779750065845]
522
[0.032706676550754016, 0.032506680533707]
523
[0.032941020690540175, 0.03228512669649385]
524
[0.03150008100269071, 0.03157838722808412]
525
[0.032232027192631824, 0.03155304803629835]
526
[0.03138742738495747, 0.03285280848177858]
527
[0.03229093168711826, 0.03206300626976698]
528
[0.03159381297163017, 0.03231022666092312]
529
[0.032194387531836566, 0.0325795634269614]
530
[0.03234789074294333, 0.03226047825558989]
531
[0.03298531913862516, 0.031607161243662]
532
[0.03233252184569343, 0.0326985472619205]
533
[0.03273685614682434, 0.03164519706304946]
534
[0.03187257717977401, 0.03236411039865177]
535
[0.03

...

3165
[0.0223332099237273, 0.009681634031190538]
3166
[0.020900726387524216, 0.008929593371327607]
3167
[0.021390366069755896, 0.009150113869973903]
3168
[0.021544273828503067, 0.009394509949287541]
3169
[0.021992557121415266, 0.009285035512515956]
3170
[0.020833093680266146, 0.010129157054363668]
3171
[0.021038263613625846, 0.00959364064422055]
3172
[0.022454713734951854, 0.009645038244535723]
3173
[0.020328150734760114, 0.009344752412371652]
3174
[0.022073106039431366, 0.009708490049280047]
3175
[0.0223610253888373, 0.010092715191874477]
3176
[0.022036499220697277, 0.00947860380851928]
3177
[0.02189787255547549, 0.009454280916641689]
3178
[0.021104140883732865, 0.00945946047035123]
3179
[0.019958299552978943, 0.00931634387133409]
3180
[0.02082226940011949, 0.009588371359996725]
3181
[0.022654925858975366, 0.010404645034075939]
3182
[0.02227859505391934, 0.009367288400669166]
3183
[0.021243517356010214, 0.00941906723282435]
3184
[0.02211663357884569, 0.009285434295123341]
3185
[0.02037

3309
[0.04891553921914406, 0.010847855202707654]
3310
[0.04993648524106171, 0.006779277965597132]
3311
[0.04863383286582872, 0.007796025644393593]
3312
[0.04605180632826871, 0.007789323666314146]
3313
[0.049846702092918097, 0.01088623183344576]
3314
[0.04865505136917748, 0.007921608942718344]
3315
[0.050514123201057505, 0.009984636561003601]
3316
[0.05084161282786451, 0.010027965605087537]
3317
[0.04844342193884318, 0.006059824975714533]
3318
[0.05075614438117584, 0.007277577929504086]
3319
[0.04879184647798602, 0.004119794458345564]
3320
[0.04752013658664009, 0.006584340939338512]
3321
[0.05061860491554098, 0.007534789652075266]
3322
[0.04784895352971846, 0.004694405784238572]
3323
[0.04853119575852905, 0.005183798887387784]
3324
[0.05295648273970803, 0.007762075692318732]
3325
[0.052980794199423295, 0.010697521199929397]
3326
[0.05345624888018988, 0.007256965573468457]
3327
[0.05189343687313446, 0.008519297565934508]
3328
[0.04538241166387297, 0.013497775437481123]
3329
[0.0516387386

3480
[0.01809139297516487, 0.048064472160852176]
3481
[0.02108806593206427, 0.047062835320072785]
3482
[0.014304307025295055, 0.04708153950478492]
3483
[0.018052392734840324, 0.044074489911858486]
3484
[0.019394802142206083, 0.05070350863314092]
3485
[0.017136868021706986, 0.04813379956650208]
3486
[0.01536394396862781, 0.04809173535125559]
3487
[0.02169956282985801, 0.05032826866888207]
3488
[0.021587787783039037, 0.04958846193258885]
3489
[0.022146645746561184, 0.04991074805294316]
3490
[0.022337131535550843, 0.04726303067987848]
3491
[0.015533713669327156, 0.047622197925586286]
3492
[0.019211021337019997, 0.048733934959440575]
3493
[0.01677682095190883, 0.05093333928534096]
3494
[0.024118116971798515, 0.049443008126490086]
3495
[0.019109963996980072, 0.04634899116117585]
3496
[0.01553024444467249, 0.049126797962252645]
3497
[0.019038147859323454, 0.04913003000045764]
3498
[0.015233140546302516, 0.05054912483249412]
3499
[0.017392624694508453, 0.05027626570958567]
3500
[0.01720186329

3654
[0.01978448316123604, 0.04074252470401172]
3655
[0.018373879971864463, 0.03963435974757076]
3656
[0.01993519654560043, 0.039658215103587635]
3657
[0.021037864302846574, 0.03871559049780586]
3658
[0.018613678674954264, 0.03915462935449883]
3659
[0.019813864407980854, 0.03850624939504124]
3660
[0.019156127218914843, 0.039397767676276986]
3661
[0.01882205313718164, 0.039917244104691205]
3662
[0.020270127492720908, 0.03876216662306714]
3663
[0.019981286325446625, 0.041152669968641364]
3664
[0.0202547733017554, 0.03707220108344741]
3665
[0.020477588298327992, 0.039109101665918106]
3666
[0.01921110792508991, 0.0400903766825955]
3667
[0.019096701548600763, 0.04055098069061758]
3668
[0.020622326018075194, 0.03945203352527068]
3669
[0.019298632594433755, 0.038201378651055905]
3670
[0.019856380340839374, 0.03893299747602459]
3671
[0.018361723178645668, 0.03833952457752689]
3672
[0.019556420622317414, 0.03839738463179272]
3673
[0.01820818082672558, 0.03603056599491631]
3674
[0.01813165521234

3824
[0.009416556239141704, 0.04085972969921637]
3825
[0.010512170294004091, 0.04373851745942099]
3826
[0.011025877580450157, 0.04140868079227225]
3827
[0.008199694912729328, 0.04103521271363801]
3828
[0.008011491586318344, 0.038005635334472765]
3829
[0.013218162865479756, 0.04234993926626538]
3830
[0.00849206487896506, 0.04025376792948044]
3831
[0.013313170903056585, 0.0413457252870204]
3832
[0.012005062335702208, 0.040845320665032596]
3833
[0.011589922116965632, 0.0363619639785288]
3834
[0.01085477327261792, 0.040441903867286445]
3835
[0.014241217018697665, 0.04105388543792837]
3836
[0.013288726466403616, 0.039340197164997606]
3837
[0.009448534318338675, 0.03890887969240239]
3838
[0.009683854303857386, 0.039061484326400754]
3839
[0.011638568006281614, 0.03963551367928871]
3840
[0.013999171042946398, 0.040849527628811784]
3841
[0.012064635979634277, 0.04149812731175116]
3842
[0.013512465132866235, 0.04044190905118849]
3843
[0.01449955069787538, 0.03804989235456636]
3844
[0.01277998281